In [5]:
import os
import pandas as pd
import numpy as np
from math import comb

def pass_at_k(results: np.ndarray, k: int) -> float:
    n, m = results.shape
    if k > n:
        return np.nan
    sample_results = results.T
    total_pass = 0.0
    for i in range(m):
        successes = int(np.sum(sample_results[i]))
        failures = n - successes
        if successes >= k:
            total_pass += 1.0
        elif failures >= k:
            fail_comb = comb(failures, k)
            total_comb = comb(n, k)
            total_pass += 1.0 - (fail_comb / total_comb)
        else:
            total_pass += 1.0
    return total_pass / m

def load_csvs_from_dir(dir_path: str) -> np.ndarray:
    csv_files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]
    if not csv_files:
        raise ValueError("No CSV files")
    
    all_data = []
    for fname in sorted(csv_files):
        df = pd.read_csv(os.path.join(dir_path, fname))
        if 'global_index' not in df.columns or 'score' not in df.columns:
            raise ValueError(f"Missing columns in {fname}")
        df['score'] = pd.to_numeric(df['score'], errors='coerce').fillna(0).astype(int)
        df = df[['global_index', 'score']].set_index('global_index')
        all_data.append(df)
    
    all_indices = sorted(set(idx for df in all_data for idx in df.index))
    aligned = [df.reindex(all_indices, fill_value=0)['score'].values for df in all_data]
    return np.array(aligned)

def extract_metadata_robust(leaf_dir: str):
    """
    从叶子目录路径中提取 model, dataset, temperature
    特殊处理 AIME2025-I / AIME2025-II
    """
    parts = os.path.normpath(leaf_dir).split(os.sep)
    
    # 初始化
    model = dataset = temp_str = "unknown"

    # 查找 AIME2025 特殊结构
    aime_handled = False
    for i in range(len(parts) - 1):
        if parts[i] == "AIME2025":
            if i + 1 < len(parts):
                next_part = parts[i + 1]
                if next_part in ("AIME2025-I", "AIME2025-II"):
                    # 找到子数据集
                    dataset = next_part
                    # model 是 AIME2025 的上一级
                    if i - 1 >= 0:
                        model = parts[i - 1]
                    # temperature 在子数据集之后
                    if i + 2 < len(parts):
                        temp_candidate = parts[i + 2]
                        # 判断是否为温度（可能是数字或 range）
                        # 如果下下级像 range（如 0-100），则再下一级才是温度？
                        # 但通常结构是: .../model/AIME2025/AIME2025-I/1.0/[0-100]/
                        # 所以温度在 i+2
                        temp_str = temp_candidate
                    else:
                        temp_str = "unknown"
                    aime_handled = True
                    break

    if not aime_handled:
        # 普通逻辑：尝试从末尾提取
        if len(parts) >= 4:
            last_part = parts[-1]
            # 判断最后一级是否为 range（含数字和 -）
            if '-' in last_part and any(c.isdigit() for c in last_part):
                # 可能是 range 目录，跳过
                if len(parts) >= 5:
                    model = parts[-4]
                    dataset = parts[-3]
                    temp_str = parts[-2]
                else:
                    model = parts[-3]
                    dataset = parts[-2]
                    temp_str = parts[-1]
            else:
                # 最后一级不是 range
                model = parts[-3]
                dataset = parts[-2]
                temp_str = parts[-1]
        elif len(parts) >= 3:
            model = parts[-3]
            dataset = parts[-2]
            temp_str = parts[-1]
        else:
            model = dataset = temp_str = "unknown"

    # 解析 temperature
    try:
        temperature = float(temp_str)
    except (ValueError, TypeError):
        temperature = temp_str

    return model, dataset, temperature


In [2]:
root_dir = "results/Qwen3-0.6B/MATH-500"
output_csv = "qwen500.csv"
rows = []

# 遍历所有目录，找出包含 .csv 的叶子目录
for dirpath, dirnames, filenames in os.walk(root_dir):
    csv_files = [f for f in filenames if f.endswith('.csv')]
    if not csv_files:
        continue

    print(f"处理: {dirpath}")
    try:
        results = load_csvs_from_dir(dirpath)
        n_runs, n_samples = results.shape
        overall_acc = float(np.mean(results))

        model, dataset, temperature = extract_metadata_robust(dirpath)

        # 计算 pass@k
        pass1 = pass_at_k(results, 1)
        pass3 = pass_at_k(results, 3) if n_runs >= 3 else np.nan
        pass5 = pass_at_k(results, 5) if n_runs >= 5 else np.nan
        pass10 = pass_at_k(results, 10) if n_runs >= 10 else np.nan

        rows.append({
            "model": model,
            "dataset": dataset,
            "temperature": temperature,
            "num_runs": n_runs,
            "num_samples": n_samples,
            "overall_accuracy": round(overall_acc, 6),
            "pass@1": round(pass1, 6),
            "pass@3": round(pass3, 6) if not np.isnan(pass3) else None,
            "pass@5": round(pass5, 6) if not np.isnan(pass5) else None,
            "pass@10": round(pass10, 6) if not np.isnan(pass10) else None,
        })

    except Exception as e:
        print(f"⚠️ 跳过 {dirpath}: {e}")
        continue

# 保存
if rows:
    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"\n✅ 成功汇总 {len(df)} 个实验结果到 {output_csv}")
else:
    print("❌ 未找到任何有效实验目录")

处理: results/Qwen3-0.6B/MATH-500
⚠️ 跳过 results/Qwen3-0.6B/MATH-500: Missing columns in run1.csv
处理: results/Qwen3-0.6B/MATH-500/0.4
处理: results/Qwen3-0.6B/MATH-500/0.4/0-500
处理: results/Qwen3-0.6B/MATH-500/0.6/0-500
处理: results/Qwen3-0.6B/MATH-500/0.8
⚠️ 跳过 results/Qwen3-0.6B/MATH-500/0.8: Missing columns in run1.csv
处理: results/Qwen3-0.6B/MATH-500/1.0/0-500
处理: results/Qwen3-0.6B/MATH-500/1.2

✅ 成功汇总 5 个实验结果到 qwen500.csv


In [7]:
import os
import pandas as pd
import numpy as np
from math import comb

# ----------------------------
# 配置路径和参数
# ----------------------------
BASE_DIR = "results/Qwen3-0.6B/MATH-500"
TEMPERATURES = [0.4, 0.6, 0.8, 1.0, 1.2]
OUTPUT_CSV = "MATH-500_passk.csv"

# ----------------------------
# 工具函数：计算 pass@k
# ----------------------------

def pass_at_k(results: np.ndarray, k: int) -> float:
    n, m = results.shape
    if k > n:
        return float('nan')
    total_pass = 0.0
    for i in range(m):
        successes = int(np.sum(results[:, i]))
        failures = n - successes
        if successes >= k:
            total_pass += 1.0
        elif failures >= k:
            # 所有 k 个都失败的概率
            prob_all_fail = comb(failures, k) / comb(n, k)
            total_pass += 1.0 - prob_all_fail
        else:
            total_pass += 1.0  # 不可能全失败
    return total_pass / m

# ----------------------------
# 主程序
# ----------------------------
all_runs_data = []  # 存放每个 run 的 (global_index, score)

for temp in TEMPERATURES:
    temp_str = str(temp)
    temp_path = os.path.join(BASE_DIR, temp_str)
    
    if not os.path.exists(temp_path):
        raise FileNotFoundError(f"❌ Temperature directory not found: {temp_path}")
    
    print(f"🔍 Scanning temperature {temp} at {temp_path}")
    
    # 递归查找该温度下所有 .csv 文件
    csv_files = []
    for root, _, files in os.walk(temp_path):
        for f in files:
            if f.endswith('.csv'):
                csv_files.append(os.path.join(root, f))
    
    if not csv_files:
        raise ValueError(f"❌ No CSV files found under {temp_path}")
    
    print(f"   Found {len(csv_files)} CSV files.")
    
    # 读取每个 CSV
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
            if 'global_index' not in df.columns or 'score' not in df.columns:
                raise ValueError(f"File {csv_file} missing 'global_index' or 'score'")
            # 转换 score 为整数（0/1）
            df['score'] = pd.to_numeric(df['score'], errors='coerce').fillna(0).astype(int)
            # 只保留需要的列
            all_runs_data.append(df[['global_index', 'score']].copy())
        except Exception as e:
            print(f"⚠️ Skipping file {csv_file}: {e}")
            continue

print(f"\n✅ Total runs loaded: {len(all_runs_data)}")

if len(all_runs_data) == 0:
    raise RuntimeError("No valid runs loaded!")

# 合并所有 runs
full_df = pd.concat(all_runs_data, ignore_index=True)

# 获取所有唯一 global_index 并排序（应为 0~499）
unique_indices = sorted(full_df['global_index'].unique())
print(f"📊 Unique samples: {len(unique_indices)} (min={min(unique_indices)}, max={max(unique_indices)})")

# 构建映射：global_index → column index
index_to_col = {idx: i for i, idx in enumerate(unique_indices)}
num_samples = len(unique_indices)
num_runs = len(full_df)

# 初始化结果矩阵 (num_runs × num_samples)
matrix = np.zeros((num_runs, num_samples), dtype=int)

# 填充矩阵
for i, (_, row) in enumerate(full_df.iterrows()):
    col = index_to_col[row['global_index']]
    matrix[i, col] = row['score']

print(f"🧮 Result matrix shape: {matrix.shape}")

# 计算指标
overall_acc = float(np.mean(matrix))
pass1 = pass_at_k(matrix, 1)
pass3 = pass_at_k(matrix, 3)
pass5 = pass_at_k(matrix, 5)
pass10 = pass_at_k(matrix, 10)
pass50 = pass_at_k(matrix, 50)

# 准备输出
result_row = {
    "dataset": "MATH-500",
    "num_runs": num_runs,
    "num_samples": num_samples,
    "overall_accuracy": round(overall_acc, 6),
    "pass@1": round(pass1, 6),
    "pass@3": round(pass3, 6),
    "pass@5": round(pass5, 6),
    "pass@10": round(pass10, 6),
    "pass@50": round(pass50, 6),
}

# 保存到 CSV
output_df = pd.DataFrame([result_row])
output_df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
print(f"\n✅ Successfully saved results to: {OUTPUT_CSV}")
print("\nResult:")
print(output_df.to_string(index=False))

🔍 Scanning temperature 0.4 at results/Qwen3-0.6B/MATH-500/0.4
   Found 10 CSV files.
🔍 Scanning temperature 0.6 at results/Qwen3-0.6B/MATH-500/0.6
   Found 10 CSV files.
🔍 Scanning temperature 0.8 at results/Qwen3-0.6B/MATH-500/0.8
   Found 10 CSV files.
🔍 Scanning temperature 1.0 at results/Qwen3-0.6B/MATH-500/1.0
   Found 10 CSV files.
🔍 Scanning temperature 1.2 at results/Qwen3-0.6B/MATH-500/1.2
   Found 10 CSV files.

✅ Total runs loaded: 50
📊 Unique samples: 500 (min=0, max=499)
🧮 Result matrix shape: (25000, 500)

✅ Successfully saved results to: MATH-500_passk.csv

Result:
 dataset  num_runs  num_samples  overall_accuracy  pass@1  pass@3   pass@5  pass@10  pass@50
MATH-500     25000          500          0.000903   0.922 0.86601 0.830042  0.75829 0.043858


In [8]:
import os
import pandas as pd
import numpy as np
from math import comb

# ----------------------------
# 配置
# ----------------------------
MODEL_ROOT = "results/Qwen3-0.6B"
TEMPERATURES = [0.4, 0.6, 0.8, 1.0, 1.2]
OUTPUT_CSV = "qwen0.6b_all_datasets_passk.csv"

# ----------------------------
# 工具函数：计算 pass@k
# ----------------------------

def pass_at_k(results: np.ndarray, k: int) -> float:
    n, m = results.shape
    if k > n:
        return float('nan')
    total_pass = 0.0
    for i in range(m):
        successes = int(np.sum(results[:, i]))
        failures = n - successes
        if successes >= k:
            total_pass += 1.0
        elif failures >= k:
            prob_all_fail = comb(failures, k) / comb(n, k)
            total_pass += 1.0 - prob_all_fail
        else:
            total_pass += 1.0
    return total_pass / m

# ----------------------------
# 加载单个数据集（或子数据集）的所有 runs
# ----------------------------

def load_dataset_runs(dataset_path: str):
    """从 dataset_path（如 .../MATH-500 或 .../AIME2025/AIME2025-I）加载所有温度下的 CSV"""
    all_dfs = []
    for temp in TEMPERATURES:
        temp_str = str(temp)
        temp_dir = os.path.join(dataset_path, temp_str)
        if not os.path.exists(temp_dir):
            print(f"  ⚠️ Temperature {temp} missing for {os.path.basename(dataset_path)}")
            continue

        # 递归找所有 .csv
        csv_files = []
        for root, _, files in os.walk(temp_dir):
            for f in files:
                if f.endswith('.csv'):
                    csv_files.append(os.path.join(root, f))
        
        if not csv_files:
            print(f"  ⚠️ No CSVs found for temp {temp}")
            continue

        # 读取每个 CSV
        for csv_file in csv_files:
            try:
                df = pd.read_csv(csv_file)
                if 'global_index' not in df.columns or 'score' not in df.columns:
                    raise ValueError(f"Missing columns in {csv_file}")
                df['score'] = pd.to_numeric(df['score'], errors='coerce').fillna(0).astype(int)
                all_dfs.append(df[['global_index', 'score']].copy())
            except Exception as e:
                print(f"    ❌ Skip {csv_file}: {e}")
    
    if not all_dfs:
        return None
    
    return pd.concat(all_dfs, ignore_index=True)

# ----------------------------
# 主程序
# ----------------------------


if not os.path.exists(MODEL_ROOT):
    raise FileNotFoundError(f"Model root not found: {MODEL_ROOT}")

# 构建要处理的数据集列表
datasets_to_process = []

# 遍历 Qwen3-0.6B 下的一级目录
for item in os.listdir(MODEL_ROOT):
    item_path = os.path.join(MODEL_ROOT, item)
    if not os.path.isdir(item_path):
        continue

    if item == "AIME2025":
        # 特殊处理：拆成两个子集
        for sub in ["AIME2025-I", "AIME2025-II"]:
            sub_path = os.path.join(item_path, sub)
            if os.path.isdir(sub_path):
                datasets_to_process.append((sub, sub_path))
            else:
                print(f"⚠️ Sub-dataset {sub} not found under AIME2025")
    else:
        # 普通数据集
        datasets_to_process.append((item, item_path))

print(f"📁 Found {len(datasets_to_process)} datasets to process:")
for name, _ in datasets_to_process:
    print(f"  - {name}")

results = []

for dataset_name, dataset_path in datasets_to_process:
    print(f"\n🚀 Processing dataset: {dataset_name}")
    full_df = load_dataset_runs(dataset_path)
    
    if full_df is None or full_df.empty:
        print(f"  ❌ No valid data for {dataset_name}, skipping.")
        continue

    # 获取唯一 global_index 并排序
    unique_indices = sorted(full_df['global_index'].unique())
    index_to_col = {idx: i for i, idx in enumerate(unique_indices)}
    num_samples = len(unique_indices)
    num_runs = len(full_df)

    # 构建矩阵
    matrix = np.zeros((num_runs, num_samples), dtype=int)
    for i, (_, row) in enumerate(full_df.iterrows()):
        col = index_to_col[row['global_index']]
        matrix[i, col] = row['score']

    print(f"  → Matrix shape: ({num_runs}, {num_samples})")

    # 计算指标
    overall_acc = float(np.mean(matrix))
    pass1 = pass_at_k(matrix, 1)
    pass3 = pass_at_k(matrix, 3)
    pass5 = pass_at_k(matrix, 5)
    pass10 = pass_at_k(matrix, 10)
    pass50 = pass_at_k(matrix, 50)

    results.append({
        "dataset": dataset_name,
        "num_runs": num_runs,
        "num_samples": num_samples,
        "overall_accuracy": round(overall_acc, 6),
        "pass@1": round(pass1, 6),
        "pass@3": round(pass3, 6),
        "pass@5": round(pass5, 6),
        "pass@10": round(pass10, 6),
        "pass@50": round(pass50, 6),
    })

# 保存结果
if results:
    df_out = pd.DataFrame(results)
    df_out.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
    print(f"\n✅ Successfully saved {len(results)} results to {OUTPUT_CSV}")
    print("\nSummary:")
    print(df_out.to_string(index=False))
else:
    print("❌ No results generated.")

📁 Found 5 datasets to process:
  - AIME2025-I
  - AIME2025-II
  - aime_2024
  - HMMT_2025
  - MATH-500

🚀 Processing dataset: AIME2025-I
  → Matrix shape: (630, 15)

🚀 Processing dataset: AIME2025-II
  → Matrix shape: (630, 15)

🚀 Processing dataset: aime_2024
  → Matrix shape: (1500, 30)

🚀 Processing dataset: HMMT_2025
  → Matrix shape: (1230, 30)

🚀 Processing dataset: MATH-500
  → Matrix shape: (25000, 500)

✅ Successfully saved 5 results to qwen0.6b_all_datasets_passk.csv

Summary:
    dataset  num_runs  num_samples  overall_accuracy   pass@1   pass@3   pass@5  pass@10  pass@50
 AIME2025-I       630           15          0.007831 0.533333 0.466984 0.271927 0.277113 0.239890
AIME2025-II       630           15          0.004656 0.200000 0.200000 0.200000 0.138475 0.129466
  aime_2024      1500           30          0.000844 0.333333 0.200333 0.134664 0.008329 0.039224
  HMMT_2025      1230           30          0.001572 0.266667 0.100407 0.100678 0.069357 0.055224
   MATH-500     25